In [1]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.signal import find_peaks
from scipy.integrate import trapz


# HRV

In [9]:
# Load RR intervals from the file
rr_intervals = pd.read_csv('PCS20_V1_RRIntervals.csv')  # Replace with your file name
rr_intervals = rr_intervals[' rr'].values  # Assuming the column is named 'RR_intervals'

# Calculate SDNN
sdnn = np.std(rr_intervals)

# Calculate RMSSD
differences = np.diff(rr_intervals)
rmssd = np.sqrt(np.mean(differences ** 2))

# Calculate Heart Rate
heart_rate = 60000 / np.mean(rr_intervals)  # Assuming RR intervals are in milliseconds

# Calculate LF and HF
time = np.cumsum(rr_intervals) / 1000.0  # Convert to seconds
fs = 4.0  # Sampling frequency (Hz)
f, psd = welch(rr_intervals, fs=fs, nperseg=1024)
lf_band = (0.04, 0.15)  # Low-frequency band
hf_band = (0.15, 0.4)   # High-frequency band
lf_power = np.trapz(psd[(f >= lf_band[0]) & (f <= lf_band[1])], f[(f >= lf_band[0]) & (f <= lf_band[1])])
hf_power = np.trapz(psd[(f >= hf_band[0]) & (f <= hf_band[1])], f[(f >= hf_band[0]) & (f <= hf_band[1])])
total_power = lf_power + hf_power
lf_percent = (lf_power / total_power) * 100
hf_percent = (hf_power / total_power) * 100
lf_hf_ratio = lf_power / hf_power

print(f"SDNN: {sdnn:.2f} ms")
print(f"RMSSD: {rmssd:.2f} ms")
print(f"Heart Rate: {heart_rate:.2f} bpm")
print(f"LF%: {lf_percent:.2f}%")
print(f"HF%: {hf_percent:.2f}%")
print(f"LF/HF Ratio: {lf_hf_ratio:.2f}")

SDNN: 58.47 ms
RMSSD: 44.07 ms
Heart Rate: 57.87 bpm
LF%: 52.25%
HF%: 47.75%
LF/HF Ratio: 1.09


c:\ProgramData\anaconda3\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 1024 is greater than input length  = 651, using nperseg = 651
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


# CPET

## Anaerobic Threshold

In [ ]:

# Import CPET data for anaerobic threshold analysis
df_cpet = pd.read_csv("PCS13_V2_CPET.csv", header=0)

# Remove any spaces in the names of the columns
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# Display the first few rows
print(df_cpet.head())


# Assuming df_cpet has columns: 'VO2', 'VCO2', 'VE', 'VE/VO2', 'VE/VCO2'
# calculate the first derivative of VE/VO2 and VE/VCO2 to find the inflection points.

# Calculate the derivative of VE/VO2 and VE/VCO2
df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE/VO2'])
df_cpet['dVE_VCO2'] = np.gradient(df_cpet['VE/VCO2'])

# Find peaks in the derivative which correspond to the anaerobic threshold
peaks_VO2 = find_peaks(df_cpet['dVE_VO2'])[0]
peaks_VCO2 = find_peaks(df_cpet['dVE_VCO2'])[0]

# Assuming the anaerobic threshold is at the first peak (can be adjusted based on the data)
at_VO2 = df_cpet['VO2'].iloc[peaks_VO2[0]] if len(peaks_VO2) > 0 else None

print(f"Anaerobic Threshold (VO2): {at_VO2}")


       TIME    VO2/kg       VO2      VCO2         VE       RER   O2pulse  \
0  0.504333  6.749663  0.428604  0.306850  12.822683  0.715930  5.357545   
1  1.034667  7.793504  0.494888  0.353991  13.592973  0.715297  5.264760   
2  1.529333  9.221128  0.585542  0.399994  14.951139  0.683117  6.036511   
3  2.010000  8.979329  0.570187  0.400834  15.153767  0.702986  6.131047   
4  2.519834  9.242735  0.586914  0.419150  15.753556  0.714159  6.521263   

          RR        Vt    HR  WorkR  HR.1    VE/VCO2     VE/VO2     PetCO2  
0  19.828157  0.646691  80.0      0  80.0  41.788074  29.917349  22.530447  
1  20.741674  0.655346  94.0      0  94.0  38.399170  27.466793  24.193222  
2  22.237198  0.672348  97.0      0  97.0  37.378441  25.533863  24.837986  
3  22.884882  0.662174  93.0      0  93.0  37.805630  26.576818  24.571787  
4  25.498529  0.617822  90.0      0  90.0  37.584572  26.841352  24.729916  
Anaerobic Threshold (VO2): 0.57018739


# V2

In [6]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression

# Import and clean data
df_cpet = pd.read_csv("WAVILA_V1_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')
print(df_cpet.head())

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Find row with peak VO2 for parameter calculations
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])  # assume VO2 is already in L/min
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None

# Anaerobic Threshold Calculation (with %VO2peak)
try:
    if all(col in df_cpet.columns for col in ['VE/VO2', 'VE/VCO2', 'VO2']):
        df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE/VO2'])
        df_cpet['dVE_VCO2'] = np.gradient(df_cpet['VE/VCO2'])
        
        peaks_VO2 = find_peaks(df_cpet['dVE_VO2'])[0]
        if len(peaks_VO2) > 0:
            at_vo2 = df_cpet['VO2'].iloc[peaks_VO2[0]]
            results['Anaerobic Threshold (VO2)'] = f"{at_vo2:.2f} L/min"
            if vo2_peak_abs and vo2_peak_abs > 0:
                at_pct = (at_vo2 / vo2_peak_abs) * 100
                results['Anaerobic Threshold (%VO2peak)'] = f"{at_pct:.1f} %"
        else:
            results['Anaerobic Threshold (VO2)'] = "Could not determine"
    else:
        missing = [col for col in ['VE/VO2', 'VE/VCO2', 'VO2'] if col not in df_cpet.columns]
        results['Anaerobic Threshold (VO2)'] = f"Missing columns: {', '.join(missing)}"
except Exception as e:
    results['Anaerobic Threshold (VO2)'] = f"Calculation error: {str(e)}"

# VO2 Peak Calculations
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# Parameter calculations at peak VO2
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'),
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# Calculate O2_Pulse if not in original data but we have VO2 and HR
if 'O2_Pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2', 'HR']):
    df_cpet['O2_Pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2'] / df_cpet['HR'], np.nan)

# Calculate OUES (Oxygen Uptake Efficiency Slope)
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values  # already in L/min
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)

        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# Print formatted results
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")


       TIME    VO2/kg       VO2      VCO2         VE       RER   O2pulse  \
0  0.509333  2.321671  0.169482  0.124037   7.932011  0.731862  2.607415   
1  1.158667  0.991074  0.072348  0.049030   2.103772  0.677697  1.079827   
2  2.269833  0.358038  0.026137  0.017141   0.861701  0.655805  0.387211   
3  2.531167  5.428626  0.396290  0.268434  10.909323  0.677369  5.743329   
4  3.114166  1.415353  0.103321  0.071940   3.015087  0.696274  1.508333   

          RR        Vt    HR  WorkR  HR.1    VE/VCO2     VE/VO2     PetCO2  
0  17.670158  0.448893  65.0      0  65.0  63.948505  46.801498  14.860900  
1   7.700205  0.273210  67.0      0  67.0  42.907600  29.078348  21.759830  
2   1.799910  0.478747  67.5      0  67.5  50.272488  32.968952  23.349535  
3  15.306122  0.712742  69.0      0  69.0  40.640575  27.528658  22.975996  
4   6.861063  0.439449  68.5      0  68.5  41.911362  29.181803  23.084255  

CPET Analysis Results:
Anaerobic Threshold (%VO2peak) : 1.7 %
Anaerobic Threshol

## BEST VERSION

In [8]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression

# Import and clean data
df_cpet = pd.read_csv("PCS20_V1_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')
print(df_cpet.head())

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Find row with peak VO2 for parameter calculations
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])  # already assumed to be in L/min
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None

# Updated Anaerobic Threshold Calculation using VO2 filter
try:
    if all(col in df_cpet.columns for col in ['VE/VO2', 'VE/VCO2', 'VO2']):
        df_cpet['VE_VO2_smoothed'] = df_cpet['VE/VO2'].rolling(window=5, center=True).mean()
        df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE_VO2_smoothed'])

        # Only consider rows where VO2 is physiologically reasonable (e.g., >1.0 L/min)
        valid_indices = df_cpet[df_cpet['VO2'] > 1.0].index
        peaks = find_peaks(df_cpet['dVE_VO2'].iloc[valid_indices])[0]

        if len(peaks) > 0:
            peak_index = valid_indices[peaks[0]]
            at_vo2 = df_cpet['VO2'].iloc[peak_index]
            results['Anaerobic Threshold (VO2)'] = f"{at_vo2:.2f} L/min"
            if vo2_peak_abs > 0:
                at_pct = (at_vo2 / vo2_peak_abs) * 100
                results['Anaerobic Threshold (%VO2peak)'] = f"{at_pct:.1f} %"
        else:
            results['Anaerobic Threshold (VO2)'] = "Could not determine"
    else:
        missing = [col for col in ['VE/VO2', 'VE/VCO2', 'VO2'] if col not in df_cpet.columns]
        results['Anaerobic Threshold (VO2)'] = f"Missing columns: {', '.join(missing)}"
except Exception as e:
    results['Anaerobic Threshold (VO2)'] = f"Calculation error: {str(e)}"

# VO2 Peak Calculations
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# Parameters at Peak VO2
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'), 
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# O2 Pulse (calculated if missing)
if 'O2_Pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2', 'HR']):
    df_cpet['O2_Pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2'] / df_cpet['HR'], np.nan)

# OUES Calculation
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)
        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# Output
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")


       TIME     VO2/kg       VO2      VCO2         VE       RER    O2pulse  \
0  0.505667   5.061225  0.491445  0.392834  21.769384  0.799345   7.560691   
1  1.008167   4.754624  0.461674  0.329129  17.583191  0.712904   7.102677   
2  1.515501  10.843358  1.052890  0.793012  37.459782  0.753176  15.952880   
3  2.004001  10.474047  1.017030  0.749873  35.458286  0.737317  14.739564   
4  2.501001   8.697004  0.844479  0.637168  30.744339  0.754510  11.894072   

          RR        Vt    HR  WorkR  HR.1    VE/VCO2     VE/VO2     PetCO2  
0  33.618984  0.647532  65.0      0  65.0  55.416237  44.296688  16.905128  
1  35.820900  0.490864  65.0      0  65.0  53.423355  38.085728  17.902117  
2  41.392902  0.904981  66.0      0  66.0  47.237358  35.578056  19.649120  
3  49.129990  0.721724  69.0      0  69.0  47.285702  34.864548  19.587336  
4  40.241447  0.763997  71.0      0  71.0  48.251572  36.406277  19.562090  

CPET Analysis Results:
Anaerobic Threshold (%VO2peak) : 67.1 %
Anaer

# FMD

In [15]:
# Import the FMD dataset as df_fmd

filename = "PCS04_V2_FMD.csv"

df_fmd2 = pd.read_csv(filename, engine='python')


df_fmd = pd.read_csv(filename, skiprows=7, skipfooter=2, engine='python')

df_fmd.head()

,Time [min:sec],Mean Diameter [mm],Positive Shear Rate [sec-1],Negative Shear Rate [sec-1],Positive Velocity [cm/sec],Negative Velocity [cm/sec],Unnamed: 6,Time [ms],Instant Diameter [mm],Mean Diameter [mm].1,Positive Velocity [cm/sec].1,Negative Velocity [cm/sec].1
0,00:00,4.270,308.9,-61.9,32.97,-6.61,NaN,0,4.273,4.270,32.97,-6.61
1,00:01,4.270,284.2,-68.8,30.34,-7.34,NaN,33,4.271,4.270,32.88,-6.64
2,00:02,4.269,268.7,-68.9,28.67,-7.35,NaN,66,4.278,4.270,32.82,-6.65
3,00:03,4.269,267.8,-70.4,28.58,-7.51,NaN,100,4.279,4.271,32.78,-6.67
4,00:04,4.268,262.1,-70.2,27.96,-7.49,NaN,133,4.275,4.271,32.70,-6.70


In [16]:
# Convert 'Time [min:sec]' to total seconds
df_fmd[['Minutes', 'Seconds']] = df_fmd['Time [min:sec]'].str.split(':', expand=True).astype(float)
df_fmd['Time_seconds'] = df_fmd['Minutes'] * 60 + df_fmd['Seconds']
df_fmd.drop(columns=['Minutes', 'Seconds'], inplace=True)

# Remove rows with missing values in key columns
df_fmd_clean = df_fmd.dropna(subset=['Mean Diameter [mm]', 'Positive Shear Rate [sec-1]', 
                                      'Negative Shear Rate [sec-1]', 'Positive Velocity [cm/sec]', 
                                      'Negative Velocity [cm/sec]'])

# Define baseline and hyperemia periods based on time
baseline_period = df_fmd_clean[df_fmd_clean['Time_seconds'] < 60]  # First 60 sec
hyperemia_period = df_fmd_clean[(df_fmd_clean['Time_seconds'] >= 420) & (df_fmd_clean['Time_seconds'] < 480)]  # 7 to 8 min

# Diameter Baseline
diameter_baseline = baseline_period['Mean Diameter [mm]'].mean() / 10  # Convert mm to cm

# Diameter Maximum
df_fmd['Mean Diameter [mm]'] = pd.to_numeric(df_fmd['Mean Diameter [mm]'], errors='coerce')
diameter_max = hyperemia_period['Mean Diameter [mm]'].max() / 10  # Convert mm to cm

# FMD Percent
fmd_percent = ((diameter_max - diameter_baseline) / diameter_baseline) * 100

# Shear Rate
shear_rate_baseline = df_fmd2['Positive Shear Rate Baseline [sec-1]'][0]
shear_rate_maximum = df_fmd2['Positive Shear Rate Maximum [sec-1]'][0]
shear_rate_areatomaximum = df_fmd2['Positive Shear Rate Area to Maximum []'][0]

# Hyperemia Shear Rate Max AUC
shear_rate_auc = trapz(4 * hyperemia_period['Positive Velocity [cm/sec]'] / hyperemia_period['Mean Diameter [mm]'])

# Mean Velocity
velocity_baseline = df_fmd2['Positive Velocity Baseline [cm/sec]'][0]
velocity_maximum = df_fmd2['Positive Velocity Maximum [cm/sec]'][0]
velocity_mean = df_fmd['Positive Velocity [cm/sec]'].mean()

# Flow Rate
flow_rate = (3.1416 * ((df_fmd['Mean Diameter [mm]'] / 10) / 2) ** 2 * df_fmd['Positive Velocity [cm/sec]'] * 60).mean()

# FMD Normalized
fmd_normalized = fmd_percent / shear_rate_auc

print(f"Diameter Baseline (cm): {diameter_baseline:.3f}")
print(f"Diameter Maximum (cm): {diameter_max:.3f}")
print(f"FMD (%): {fmd_percent:.2f}")
print(f"Shear Rate Baseline (s⁻¹): {shear_rate_baseline:.2f}")
print(f"Shear Rate Maximum (s⁻¹): {shear_rate_maximum:.2f}")
print(f"Shear Rate Area to Maximum (s⁻¹): {shear_rate_areatomaximum:.2f}")
print(f"Hyperemia Shear Rate Max (AUC): {shear_rate_auc:.2f}")
print(f"Baseline Velocity (cm/s): {velocity_baseline:.2f}")
print(f"Maximum Velocity (cm/s): {velocity_maximum:.2f}")
print(f"Mean Velocity (cm/s): {velocity_mean:.2f}")
print(f"Flow Rate (ml/min): {flow_rate:.2f}")
print(f"FMD Normalized: {fmd_normalized:.5f}")


Diameter Baseline (cm): 0.427
Diameter Maximum (cm): 0.475
FMD (%): 11.32
Shear Rate Baseline (s⁻¹): 325.15
Shear Rate Maximum (s⁻¹): 894.22
Shear Rate Area to Maximum (s⁻¹): 19562.50
Hyperemia Shear Rate Max (AUC): 2197.76
Baseline Velocity (cm/s): 34.70
Maximum Velocity (cm/s): 97.44
Mean Velocity (cm/s): 29.97
Flow Rate (ml/min): 282.98
FMD Normalized: 0.00515


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_37476\1023748718.py:31: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  shear_rate_auc = trapz(4 * hyperemia_period['Positive Velocity [cm/sec]'] / hyperemia_period['Mean Diameter [mm]'])


[]
